In [63]:
import pandas as pd
import numpy as np

try:
    df_dirty = pd.read_csv('/home/bigdata/Task_1.4_Data_Wrangling_Challenge_“DirtyData”_Exercise/datacleaner/recieved_dirty_dataset.csv', encoding='latin-1')
except UnicodeDecodeError:
    # Fallback to Shift-JIS if Latin-1 fails (common for Japanese data)
    df_dirty = pd.read_csv('/home/bigdata/Task_1.4_Data_Wrangling_Challenge_“DirtyData”_Exercise/datacleaner/recieved_dirty_dataset.csv', encoding='shift-jis')

df_dirty.head()

,Rank_,Name_,platform!!!!!!!!!!!,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,8316,Hunter: The Reckoning Redeemer,XB,2003.0,Action,Interplay,0.13,ï¿½0.04,0.00,0.01,0.17
1,13417,Garou Densetsu 3: Road to the Final Victory,SAT,1996.0,Fighting,SNK,0.00,ï¿½0.00,0.05,0.00,0.05
2,15755,Zero no Tsukaima: Muma ga Tsumugu Yokaze no No...,PS2,2007.0,Adventure,Marvelous Interactive,0.00,ï¿½0.00,0.02,0.00,0.02
3,8684,MX 2002 Featuring Ricky Carmichael,GBA,2001.0,Racing,THQ,0.11,ï¿½0.04,0.00,0.00,0.16
4,14586,Battle vs. Chess,PC,NaN,Misc,TopWare Interactive,0.00,0.02,0.00,0.01,0.03


In [64]:
df_clean = df_dirty.copy()
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240 entries, 0 to 239
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Rank_                240 non-null    object 
 1   Name_                240 non-null    object 
 2   platform!!!!!!!!!!!  240 non-null    object 
 3   Year                 235 non-null    object 
 4   Genre                240 non-null    object 
 5   Publisher            237 non-null    object 
 6   NA_Sales             240 non-null    float64
 7   EU_Sales             240 non-null    object 
 8   JP_Sales             240 non-null    float64
 9   Other_Sales          240 non-null    float64
 10  Global_Sales         240 non-null    float64
dtypes: float64(4), object(7)
memory usage: 20.8+ KB


In [65]:
# CORRECCIÓN DE ENCABEZADOS (HEADERS)
df_clean.rename(columns={
'Rank_': 'Rank',
    'Name_': 'Name',
    'platform!!!!!!!!!!!': 'Platform'
}, inplace=True)

In [66]:
# ELIMINACIÓN DE DUPLICADOS
registros_iniciales = len(df_clean)
df_clean.drop_duplicates(inplace=True)
registros_finales = len(df_clean)
print(f"Registros eliminados: {registros_iniciales - registros_finales}")

Registros eliminados: 16


In [67]:
## LIMPIEZA Y CONSOLIDACIÓN DE COLUMNAS CON TIPO OBJECT
df_clean.head()

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,8316,Hunter: The Reckoning Redeemer,XB,2003.0,Action,Interplay,0.13,ï¿½0.04,0.00,0.01,0.17
1,13417,Garou Densetsu 3: Road to the Final Victory,SAT,1996.0,Fighting,SNK,0.00,ï¿½0.00,0.05,0.00,0.05
2,15755,Zero no Tsukaima: Muma ga Tsumugu Yokaze no No...,PS2,2007.0,Adventure,Marvelous Interactive,0.00,ï¿½0.00,0.02,0.00,0.02
3,8684,MX 2002 Featuring Ricky Carmichael,GBA,2001.0,Racing,THQ,0.11,ï¿½0.04,0.00,0.00,0.16
4,14586,Battle vs. Chess,PC,NaN,Misc,TopWare Interactive,0.00,0.02,0.00,0.01,0.03


In [68]:
df_clean['EU_Sales'] = df_clean['EU_Sales'].astype(str).str.replace(r'[^\d\.]', '', regex=True)
df_clean['EU_Sales'] = pd.to_numeric(df_clean['EU_Sales'], errors='coerce')
# Identificamos outliers por unidad (valores artificialmente grandes, > 35)
umbral_error_unidad = 35.0
df_clean.loc[df_clean['EU_Sales'] > umbral_error_unidad, 'EU_Sales'] /= 1000

# Imputar posibles NaN con la mediana
df_clean['EU_Sales'].fillna(df_clean['EU_Sales'].median(), inplace=True)


/tmp/ipykernel_77427/2663438113.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_clean['EU_Sales'].fillna(df_clean['EU_Sales'].median(), inplace=True)


In [69]:
# columna year
# Esto se hace asumiendo que el proceso de ensuciamiento introdujo este tipo de string.
df_clean['Year'] = df_clean['Year'].replace('Unknown', np.nan)
df_clean['Year'] = pd.to_numeric(df_clean['Year'], errors='coerce')
# Reemplazamos años futuros o irreales (> 2020) con NaN para su imputación.
df_clean.loc[df_clean['Year'] > 2025, 'Year'] = np.nan

# Imputar los valores NaN restantes (originales + corregidos) con la mediana.
mediana_year = df_clean['Year'].median()
df_clean['Year'].fillna(mediana_year, inplace=True)

#  Corregir el tipo de dato a entero (eliminando posibles decimales).
df_clean['Year'] = df_clean['Year'].astype(int)
print(f"Año Mínimo Detectado: {df_clean['Year'].min()}")
print(f"Año Máximo Detectado: {df_clean['Year'].max()}")

Año Mínimo Detectado: 1981
Año Máximo Detectado: 2016


/tmp/ipykernel_77427/2099908245.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_clean['Year'].fillna(mediana_year, inplace=True)


In [73]:
# Name
print(df_clean['Name'].head(20).to_string())

# b) Verificación de espacios extremos
initial_unique_count = df_clean['Name'].nunique()

0                        Hunter: The Reckoning Redeemer
1           Garou Densetsu 3: Road to the Final Victory
2     Zero no Tsukaima: Muma ga Tsumugu Yokaze no No...
3                    MX 2002 Featuring Ricky Carmichael
4                                      Battle vs. Chess
5                                 NASCAR Heat Evolution
6                  Motor Trend presents Lotus Challenge
7                             Tsumiki: Block Drop Mania
8                             True Crime: Streets of LA
9                            Call of Duty: World at War
10    Hisshou Pachinko*Pachi-Slot Kouryaku Series Vo...
11                                           Funky Barn
12                              Jumper: Griffin's Story
13    Tï¿½enage Mutant Ninja Turtles 3: Mutant Night...
14                                Yu-Gi-Oh! Double Pack
15                                       God of War III
16                               Penguins of Madagascar
17             Bastard!! Utsuro Naru Kamigami no

In [77]:
# Aplica strip() para eliminar espacios iniciales/finales y replace() para normalizar espacios internos.
df_clean['Name'] = df_clean['Name'].str.strip().str.replace(r'\s+', ' ', regex=True)

df_clean['Name'] = df_clean['Name'].astype(str).str.replace('ï¿½', '', regex=False)
df_clean['Name'] = df_clean['Name'].str.lower()
print(df_clean['Name'].head(10).to_string())


0                       hunter: the reckoning redeemer
1          garou densetsu 3: road to the final victory
2    zero no tsukaima: muma ga tsumugu yokaze no no...
3                   mx 2002 featuring ricky carmichael
4                                     battle vs. chess
5                                nascar heat evolution
6                 motor trend presents lotus challenge
7                            tsumiki: block drop mania
8                            true crime: streets of la
9                           call of duty: world at war


In [ ]:
#Platform, 
# Mostrar todos los valores únicos
print("--- Valores Únicos de la Columna 'Platform' ---")
plataformas_unicas = df_clean['Platform'].unique()
print(plataformas_unicas)
print(f"\nTotal de valores únicos: {len(plataformas_unicas)}")
#Observamos que no hay errores

--- Valores Únicos de la Columna 'Platform' ---
['XB' 'SAT' 'PS2' 'GBA' 'PC' 'XOne' 'DS' '3DS' 'PS3' 'Wii' 'PS' 'PSP'
 'PS4' 'X360' 'GC' 'PSV' 'N64' 'WiiU' 'SNES' '2600' 'DC' 'GB' 'NES']

Total de valores únicos: 23


In [ ]:
# Genre
# Mostrar todos los valores únicos
print("--- Valores Únicos de la Columna 'Genre' ---")
genre_unicos = df_clean['Genre'].unique()
print(genre_unicos)
print(f"\nTotal de valores únicos: {len(genre_unicos)}")
#Observamos que no hay sports esta mal escrito

--- Valores Únicos de la Columna 'Genre' ---
['Action' 'Fighting' 'Adventure' 'Racing' 'Misc' 'Shooter' 'Simulation'
 'Role-Playing' 'Sprots' 'Platform' 'Strategy' 'Puzzle']

Total de valores únicos: 12


In [86]:
df_clean['Genre'] = df_clean['Genre'].replace('Sprots', 'Sports')

In [ ]:
# Publisher
# Mostrar todos los valores únicos
print("--- Valores Únicos de la Columna 'Publisher' ---")
publisher_unicos = df_clean['Publisher'].unique()
print(publisher_unicos)
print(f"\nTotal de valores únicos: {len(publisher_unicos)}")
#Observamos que hay algun nan

--- Valores Únicos de la Columna 'Publisher' ---
['Interplay' 'SNK' 'Marvelous Interactive' 'THQ' 'TopWare Interactive'
 'Dusenberry Martin Racing' 'Xicat Interactive' 'Nobilis' 'Activision'
 'D3Publisher' 'Ubisoft' 'Eidos Interactive'
 'Konami Digital Entertainment' nan 'Namco Bandai Games'
 'Seta Corporation' 'Nippon Ichi Software' 'Introversion Software'
 'Electronic Arts' 'Idea Factory' 'PQube' 'Sony Computer Entertainment'
 'Vivendi Games' '505 Games' 'Microsoft Game Studios' 'Evolved Games'
 'Take-Two Interactive' 'Square Enix' 'Bohemia Interactive'
 'Crave Entertainment' 'Rising Star Games' 'Asylum Entertainment'
 'Banpresto' 'PERROSANXE' 'Activision Value' 'Nintendo' 'Kemco'
 'Swing! Entertainment' 'Atari' 'JoWood Productions' 'Tecmo Koei'
 'DTP Entertainment' 'Victor Interactive' 'Capcom' 'Atlus'
 'Tru Blu Entertainment' 'Warner Bros. Interactive Entertainment' 'FRIJOL'
 'Mystique' 'Sega' 'Disney Interactive Studios' 'Mindscape'
 'Telltale Games' 'DreamCatcher Interactive' 'Su

In [85]:
# Convertimos todos los valores no válidos o de error a np.nan para imputarlos juntos.
error_choices = ['PERROSANXE', 'FRIJOL', 'FACHOESFERA', 'GILIPROGRES']
df_clean['Publisher'] = df_clean['Publisher'].replace({
    "": np.nan, 
    "N/A": np.nan
} | {choice: np.nan for choice in error_choices})

# b) Imputar los valores NaN restantes con la Moda (el valor más frecuente)
# La Moda es la mejor opción para variables categóricas.
moda_publisher = df_clean['Publisher'].mode()[0]
registros_imputados = df_clean['Publisher'].isnull().sum()

df_clean['Publisher'].fillna(moda_publisher, inplace=True)
print(f"Total de valores únicos finales: {df_clean['Publisher'].nunique()}")

Total de valores únicos finales: 71


/tmp/ipykernel_77427/1323275970.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_clean['Publisher'].fillna(moda_publisher, inplace=True)


In [88]:
#Rank
print("--- Valores Únicos de la Columna 'Rank' ---")
rank_unicos = df_clean['Rank'].unique()
print(rank_unicos)
print(f"\nTotal de valores únicos: {len(rank_unicos)}")


--- Valores Únicos de la Columna 'Rank' ---
['8316' '13417' '15755' '8684' '14586' '15055' '14644th' '12983' '412'
 '2290' '12747' '10129' '11926' '12583' '6129' '217' '15024' '7421'
 '14742' '3901' '15085' '558' '4695' '3092' '15269' '16143rd' '11615'
 '9136' '11908' '11942' '4002' '12213' '6323' '12709th' '6109' '15167'
 '6594' '3254' '4952' '8857' '7780' '3590' '9891' '13920' '12524' '9745'
 '3380' '3169' '11456th' '9480' '6672' '1066' '7057' '15683' '20' '2350'
 '15366' '1210' '13010' '15544' '15912' '13312' '13701' '6183' '15881'
 '4169' '14748' '13291' '10261' '8535' '1433' '13864' '11931' '1407'
 '15663' '8127' '11111th' '13344' '13641' '8423' '15287' '1168' '11526'
 '10774' '2518' '1134' '4688' '9469' '1268' '9264' '3245th' '15773'
 '11976' '13122' '8694' '7651' '1090' '3104th' '12659' '7826' '7532'
 '14588th' '15504' '8257' '7983' '15486' '5634' '1694' '6934' '11779'
 '16246th' '3568' '1491st' '11554th' '5553' '551' '14442' '2270' '8456th'
 '6729' '12340' '431st' '4418' '16491

In [90]:
#Hay valores con strings
def clean_rank_string(rank):
    """
    Elimina los sufijos ordinales ('st', 'nd', 'rd', 'th') de los strings en 'Rank'.
    """
    if pd.notna(rank) and isinstance(rank, str):
        rank_lower = rank.lower()
        if rank_lower.endswith(('st', 'nd', 'rd', 'th')):
            return rank[:-2].replace(',', '')
        return rank.replace(',', '')
    return rank

df_clean['Rank'] = df_clean['Rank'].apply(clean_rank_string)
df_clean['Rank'] = pd.to_numeric(df_clean['Rank'], errors='coerce').astype('Int64')
print("--- Valores Únicos de la Columna 'Rank' ---")
rank_unicos = df_clean['Rank'].unique()
print(rank_unicos)
print(f"\nTotal de valores únicos: {len(rank_unicos)}")

--- Valores Únicos de la Columna 'Rank' ---
<IntegerArray>
[ 8316, 13417, 15755,  8684, 14586, 15055, 14644, 12983,   412,  2290,
 ...
  7730, 10730,   718,  2072, 14524, 13242, 13680,  1048, 15124, 10963]
Length: 200, dtype: Int64

Total de valores únicos: 200
